# Прогнозирование изменения цены акций в краткосрочном периоде

# УСТАНОВКА APIMOEX И TALIB

In [1]:
import requests # ИНТЕРНЕТ (HTTP)
import os # РАБОТА С ОПЕРАЦИОННОЙ СИСТЕМОЙ

# УСТАНОВКА APIMOEX
try:
    import apimoex
except ImportError:
    os.system(f"pip install apimoex")
    
# УСТАНОВКА TALIB
python_version = "311" # ВАША ВЕРСИЯ ПИТОНА ВИДА NNN (ЕСЛИ ВЕРСИЯ 3.11, НАДО НАПИСАТЬ 311)
try:
    import talib
except ImportError:
    print("Библиотека talib не установлена. Начало установки")
    current_directory = os.getcwd()
    file_name = "TA_Lib-0.4.28-cp" + python_version + "-cp" + python_version + "-win_amd64.whl"
    url = 'https://github.com/cgohlke/talib-build/releases/download/v0.4.28/' + file_name
    response = requests.get(url)
    with open(file_name, 'wb') as f:
        f.write(response.content)
    os.system(f"pip install {file_name}")
    if os.path.exists(file_name):
        os.remove(file_name)

# ИМПОРТ БИБЛИОТЕК

In [2]:
import apimoex # МОСКОВСКАЯ БИРЖА
import mplfinance as mpf # ФИНАНСОВЫЕ ГРАФИКИ (СВЕЧИ) #pip install --upgrade mplfinance
import talib # БИБЛИОТЕКА ТЕХНИЧЕСКОГО АНАЛИЗА
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns
warnings.filterwarnings("ignore")

# ПАРСИНГ СВЕЧЕЙ С МОСБИРЖИ

In [3]:
def get_candles(start_date: str, ticker: str, timeframe: int, filename: str) -> pd.DataFrame:
    """Функция для сохранения свечей выбранного инструмента с произвольной даты и указанного таймфрейма в файл csv
    
    Args:
        start_date (str): Начальная дата, с которой загружаются свечи. Вид: YYYY-MM-DD
        ticker (str): Тикер инструмента
        timeframe (int): Таймфрейм свечи (1 - 1M, 10 - 10M, 60 - 1H, 24 - 1D, 7 - 1W, 31 - 1 month, 4 - 1 quarter)
        filename (str): Имя csv файла, куда сохранить свечи
    """   
    with requests.Session() as session:
        if ticker == "IMOEX":
            data = apimoex.get_market_candles(session, "IMOEX", interval = timeframe, start = start_date, market = 'index')
        else:
            data = apimoex.get_market_candles(session, ticker, interval = timeframe, start = start_date)            
        candles = pd.DataFrame(data)
        candles.to_csv(filename + '.csv', index = False)
        first_candle_time = candles.iloc[1]['begin']
        last_candle_time = candles.iloc[-1]['begin']
        print(f'Время обновления: {datetime.now().replace(microsecond = 0)}')
        print(f'Количество свеч по {ticker}: {candles.shape[0]}')
        print(f'Время первой полученной свечи: {first_candle_time}')
        print(f'Время последней полученной свечи: {last_candle_time} \n')
     

In [4]:
#get_candles("2023-05-12", "SBER", 1, 'sber')
sber = pd.read_csv('sber.csv')
sber.head()

,begin,open,close,high,low,value,volume
0,2023-05-12 09:59:00,231.00,231.00,231.00,231.00,1.608568e+08,696350
1,2023-05-12 10:00:00,231.05,232.38,232.47,230.52,6.528957e+08,2819200
2,2023-05-12 10:01:00,232.38,232.00,232.38,231.53,3.482892e+08,1501950
3,2023-05-12 10:02:00,231.90,232.10,232.38,231.61,2.798282e+08,1205930
4,2023-05-12 10:03:00,232.08,231.50,232.09,230.90,2.537112e+08,1095840


In [5]:
#get_candles("2023-05-12", "IMOEX", 1, 'imoex')
imoex = pd.read_csv('imoex.csv')
imoex.head()

,begin,open,close,high,low,value,volume
0,2023-05-12 10:00:00,2570.65,2580.41,2580.55,2570.65,1.111358e+09,0
1,2023-05-12 10:01:00,2580.72,2582.12,2582.12,2579.61,6.206923e+08,0
2,2023-05-12 10:02:00,2582.33,2584.59,2584.83,2581.75,6.074696e+08,0
3,2023-05-12 10:03:00,2584.61,2580.18,2584.91,2579.25,5.306508e+08,0
4,2023-05-12 10:04:00,2580.31,2579.21,2580.72,2579.21,2.135944e+08,0
